# 問題1
## 焼肉を食べに行きたい！！

- あなたはグルメオープンのお疲れ様会で、焼肉を食べに行きたい
- 1000店舗の緯度経度情報・料理ジャンル・営業時間を持っています (CSVファイル)
- 下記条件を全て満たしたお店を探します。
	- ある駅 (緯度 : 35.6546783 / 経度 :139.7369874) から1.0km以内のお店
	- ジャンルが焼肉のお店
	- 17時に営業中 (営業開始している) のお店
- そのお店の【answer (回答項目)】が、問題1の回答です。

※ 緯度経度の座標はすべて世界測地系の座標

CSVファイル (1000店舗の緯度経度情報) は retty_gourmet_open_q1.csv

| answer (回答候補) | latitude (緯度) | longitude (経度) | category (料理ジャンル) | hour_from (営業開始時間) | hour_to (営業終了時間) | 備考 |
| --- | --- | --- | --- | --- | --- | --- |
| PRE13/ARE8/SUB802 | 35.6625840 | 139.6973305 | ラーメン | 17:00:00 | 23:00:00 | 店舗A のお店情報 |
| PRE13/ARE21/SUB2104 | 35.7799264 | 139.7524000 | 焼肉 |  18:00:00 | 26:00:00 | 店舗Bのお店情報 |
| ... | ... | ... | ... |
| PRE14/ARE31/SUB3103 | 35.5614942 | 139.5052934 | 肉バル |  18:00:00 | 20:00:00 | 店舗C のお店情報 | 

## 回答・解説
### ポイント① 緯度経度
緯度経度とは、場所を数値的に表現する方法の一つ。

- 日本測地系 : 日本独自のもの。2002年に世界測地系へ移行している
- 世界測地系 : 国際的に定められた基準となる測地系が世界測地系

ご存知の通り、地球は平面ではなく球体 (正確には楕円体) になるので、球面を気にして計算する必要がある。

今回の回答例では、Python の GeoPy とよばれるジオコーディングライブラリを利用して実装。  
GeoPy では、カーニー法とよばれる、測地線距離を求める新しいアルゴリズムを採用。  
https://geopy.readthedocs.io/en/stable/#module-geopy.distance

今回の問題に限っていくと、平面と捉えて三平方の定理を用いて2点の距離を求めて、
条件にあてはまる最小距離の店舗でも正しい回答を導けます。

### ポイント② 営業時間の取り扱い
日付型として取り扱いでも良いが、26:00:00 といった30時間制の情報取り扱いに注意。
深夜営業店舗の営業時間表記でよく使われていて、Rettyでも同様の表記を採用している。

24時以降の差分時間を翌日として取り扱うように実装するでも良いし、
今回のように営業中かどうかの判定であれば、数値型やtimestamp型で比較でもよい。

In [73]:
from geopy.distance import geodesic
import pandas as pd
import math
from datetime import datetime,timedelta,timezone

In [74]:
df = pd.read_csv("retty_gourmet_open_q1.csv")
df.head()

,answer,latitude,longitude,category,dinner_from,dinner_to
0,PRE13/ARE11/SUB1103,35.699149,139.758853,ラーメン・麺類,17:00:00,22:00:00
1,PRE13/ARE7/SUB703,35.639204,139.704520,イタリアン,18:00:00,22:00:00
2,PRE13/ARE14/SUB1405,35.656415,139.734102,すし・魚料理,17:00:00,23:00:00
3,PRE13/ARE19/SUB1902,35.587548,139.732552,焼肉,17:00:00,23:00:00
4,PRE12/ARE41/SUB4101,35.859553,139.974580,中華,16:00:00,20:00:00


ある駅の緯度経度情報 (緯度 : 35.6546783 / 経度 : 139.7369874) と、店舗との距離を計算する

## ① 緯度経度情報に GeoPy、営業時間情報に 数値型 を利用した回答方法

In [75]:
station = (35.6546783, 139.7369874)
 
answer = []
distance = []
category = []
hour_from = []
hour_to = []

for row in df.itertuples():
    answer.append(row[1])
    category.append(row[4])
    
    # 数値型に変換して配列格納
    hour_from.append(int(row[5].replace(":","")))
    hour_to.append(int(row[6].replace(":","")))

    latlng = (row[2], row[3])
    t_dis = geodesic(station, latlng).km
    distance.append(t_dis)

df_dis = pd.DataFrame(index=[], columns=['answer', 'distance', 'category', 'hour_from', 'hour_to'])
df_dis['answer'] = answer
df_dis['distance'] = distance
df_dis['category'] = category
df_dis['hour_from'] = hour_from
df_dis['hour_to'] = hour_to

# 1.5km以内 & ジャンルが焼肉 & 17時に営業中 を判定して出力
print(df_dis.query('category=="焼肉" and distance < 1.0 and hour_from <= 170000 and 170000 < hour_to'))

                  answer  distance category  hour_from  hour_to
745  PRE13/ARE14/SUB1405  0.245061       焼肉     170000   233000


## ② 緯度経度情報に 三平方定理、営業時間情報に 日付型 を利用した回答方法

In [80]:
station = (35.6546783, 139.7369874)
 
answer = []
distance = []
category = []
hour_from = []
hour_to = []

def get_distance(x1, y1, x2, y2):
    d = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return d

def get_datetime(str_datetime):
    hour, minute, second = [int(x.strip()) for x in str_datetime.split(":")]
    day = datetime.now().replace(microsecond=0,second=0,minute=0,hour=0)
    days,hour = divmod(hour,24)
    result = day + timedelta(days=days,hours=hour,minutes=minute)
    return result

for row in df.itertuples():
    answer.append(row[1])
    category.append(row[4])
    
    # datetime型に変換して配列格納
    hour_from.append(get_datetime(row[5]))
    hour_to.append(get_datetime(row[6]))

    latlng = (row[2], row[3])
    t_dis = get_distance(latlng[0], latlng[1], station[0], station[1])
    distance.append(t_dis)

df_dis = pd.DataFrame(index=[], columns=['answer', 'distance', 'category', 'hour_from', 'hour_to'])
df_dis['answer'] = answer
df_dis['distance'] = distance
df_dis['category'] = category
df_dis['hour_from'] = hour_from
df_dis['hour_to'] = hour_to

print(df_dis.sort_values('distance').query('category=="焼肉"').loc[df_dis.hour_from <= '2021-07-30 17:00:00'].loc['2021-07-30 17:00:00' <= df_dis.hour_to].iloc[[0]])

                  answer  distance category           hour_from  \
745  PRE13/ARE14/SUB1405  0.002388       焼肉 2021-07-30 17:00:00   

                hour_to  
745 2021-07-30 23:30:00  
